In [ ]:
import gc
gc.collect()

275

In [ ]:
!pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
# Google Earth Engine을 인증합니다.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=HZ1zVt_vgpSEnGhN9TKYn1sP0b9qLEYja9PVL_IAidw&tc=Gsc09hRLwTRB5ivDRLwnqO2lhdWWRAWubB0_XTPpqFg&cc=U7v8hpA0OtUIJL2XLeDHwXB9nvV_BvlWRwNI10grIDA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VOJRx4M_wXcLZQspB5Rtj080UdLc48W_cRgM1VaKqjPfyCqmVqNU1s

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
st=['2000-05-01'] # '1991-05-01', '1992-05-01', '1993-05-01', '1994-05-01', '1995-05-01', '1996-05-01', '1997-05-01', '1998-05-01', '1999-05-01',
ed=['2000-10-31'] # '1991-10-31', '1992-10-31', '1993-10-31', '1994-10-31', '1995-10-31', '1996-10-31', '1997-10-31', '1998-10-31', '1999-10-31',

In [ ]:
# 노스다코타주의 경계를 가져옵니다
north_dakota = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '29'))

In [ ]:
# MODIS NDVI 컬렉션 필터링
collection = ee.ImageCollection('MODIS/006/MOD13A2')  # MODIS NDVI 컬렉션

In [ ]:
# 각 이미지를 다운로드합니다
for i in range(len(st)):
  # 아이오와주 내의 이미지로 필터링
  filtered = collection.filterBounds(north_dakota)

  # 노스다코타주의 geometry를 얻습니다
  north_dakota_geometry = north_dakota.geometry()

  # 날짜로 필터링
  filtered = filtered.filterDate(st[i], ed[i])

  # 해상도 설정
  resolution = 1000  # 10m 해상도로 설정

  # 노스다코타주를 클립하고 reprojection하는 함수를 정의합니다
  def clip_and_reproject(image):
      return image.clip(north_dakota_geometry).reproject('EPSG:4326', None, resolution)

  # 각 이미지를 노스다코타주로 클립하고 reprojection합니다
  north_dakota_images = filtered.map(clip_and_reproject)

  # 이미지 리스트를 얻습니다
  image_list = north_dakota_images.toList(north_dakota_images.size())

  for j in range(image_list.size().getInfo()):
      image = ee.Image(image_list.get(j))
      date = image.date().format('YYYY-MM-dd').getInfo()

      task = ee.batch.Export.image.toDrive(
          image=image.select('NDVI'),
          description='NDVI_Image_Missouri_' + date,
          folder='Missouri',
          scale=resolution,  # 해상도(scale) 설정
          region=north_dakota_geometry  # 'region'에 직접 ee.Geometry 객체 사용
      )
      task.start()